<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from scipy import stats as st
from sklearn.metrics import r2_score
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_csv('/datasets/insurance.csv') 

In [3]:
df.head() # выведем первые 5 строк

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,"49,600.00",1,0
1,0,46.00,"38,000.00",1,1
2,0,29.00,"21,000.00",0,0
3,0,21.00,"41,700.00",2,0
4,1,28.00,"26,100.00",0,0


In [4]:
df.info() # информация о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
# сразу выделим признаки и целевой признак
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Так как в нашем примере признаки должны быть умножены на обратную матрицу, то при предсказании формула будет выглядить так:

$$
a_1 = X_1 w_1
$$

где $X_1 = XP$

Во время обучения при умножении обучающей выборке помноженую на матрицу будет выглядить так:

$$
w_1 = ((XP)^T (XP))^{-1} ((XP)^T) y = (P^T X^T XP)^{-1} P^T X^T y
$$

Если дальше упрощать выражение, то можем достать $P_t$ и $P$ из скобок

$$
w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

А результат обучения:

$$
w_1 = \arg\min_w MSE((XP)w, y)
$$



Добавляем новую формулу в предсказание и смотрим результат

$$
a_1 = XP P^{-1} (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y
$$

При упрощении формулы получаем, что матрица, на которую умножали, превращается в единичную матрицу и из формулы исчезает, что доказывает нам отсутствие влияния на качество модели. Теперь посмотрим на примере.

Попробуем сначала обучить линейную регрессию и вычислим ее r2.

In [6]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ (X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [7]:
def linear_fit_and_r2(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)
    print(r2_score(target, predictions))

In [8]:
# обучим и рассчитаем r2 до преобразования
linear_fit_and_r2(features, target)

0.42494550286668


Мы получили коэфициент равный 0,424. Теперь, чтобы узнать ответ на вопрос изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу, сделаем новую квадратную матрицу, умножим признаки на неё и попробуем на них обучить модель, вычислим и сравним r2.

In [9]:
P = np.random.normal(size=(4, 4))
features_new = features @ P

In [10]:
# преобразуем и рассчитаем r2 
linear_fit_and_r2(features_new, target)

0.4249455028653486


**Ответ:** R2 у обоих моделей получился одинаковый.

**Обоснование:** Так как изменяются только параметры модели, а не весы, то на качество модели умножение на матрицу практически не влияет.

## Алгоритм преобразования

**Алгоритм**

Так как мы знаем, что при умножении на обратимую матрицу признаков модель не меняется, можем зашифровать с помощью умножения модели на матрицу. Данная матрица будет являться ключем для расшифровки исходных данных.

**Обоснование**

Чтобы проверить действительно ли таким образом данные можно восстановить/зашифровать проведем проверку.

## Проверка алгоритма

In [11]:
# генерируем матрицу, наш ключ
random_matrix = np.random.normal(size=(4, 4))
inv_random_matrix = np.linalg.inv(random_matrix)
encoded_features = features @ random_matrix

In [12]:
# до преобразования r2 
linear_fit_and_r2(features, target)

0.42494550286668


In [13]:
# после преобразования r2
linear_fit_and_r2(encoded_features, target)

0.4249455028666794


In [14]:
# расшифровываем признаки и сравниваем
decoded_features = encoded_features @ inv_random_matrix
decoded_features = pd.DataFrame(decoded_features.values, columns=features.columns)
decoded_features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1.00,41.00,"49,600.00",1.00
1,0.00,46.00,"38,000.00",1.00
2,0.00,29.00,"21,000.00",0.00
3,0.00,21.00,"41,700.00",2.00
4,1.00,28.00,"26,100.00",0.00


In [15]:
# до шифрования
features.head() 

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.00,"49,600.00",1
1,0,46.00,"38,000.00",1
2,0,29.00,"21,000.00",0
3,0,21.00,"41,700.00",2
4,1,28.00,"26,100.00",0


Как мы видим, алгоритм работает. Коэффициент практически совпадает до и после преобразования, а данные зашифрованы и могут быть возвращены в исходный вид.